In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statistics as stat
import math
import pickle
import gzip
import sklearn
import graphviz
from graphviz import Source
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import export_graphviz
from sklearn.metrics import classification_report
from sklearn.metrics import mean_squared_error 
from sklearn import svm

In [2]:
import os
os.environ["PATH"] += os.pathsep + 'C:/Program Files (x86)/Graphviz2.38/bin/'

In [13]:
#read in csv files. drop unknown column
traindf = pd.read_csv('train_data.csv', engine='python')
traindf.drop(traindf.columns[traindf.columns.str.contains('unnamed',case = False)],axis = 1, inplace = True)
testdf = pd.read_csv('val_data.csv', engine='python')
testdf.drop(testdf.columns[testdf.columns.str.contains('unnamed',case = False)],axis = 1, inplace = True)

#drop the id information becuse it does not matter
traindf.drop(traindf.columns[traindf.columns.str.contains('id',case = False)],axis = 1, inplace = True)
testdf.drop(testdf.columns[testdf.columns.str.contains('id',case = False)],axis = 1, inplace = True)

#clean the date information
traindf['date'] = traindf['date'].str.replace("(T).*","").astype(int)
testdf['date'] = testdf['date'].str.replace("(T).*","").astype(int)

traindf.info()
traindf.head()


#Creates a copy of the training df so we can arrange our data and correctly format it
traindfy = traindf.copy()
traindfX = traindf.copy()
testX = testdf.copy()


#creates y and X df
y = traindfy
X = traindfX

#Shows us our data for X
X.drop(['price'], axis=1, inplace=True)
traindfX.columns = range(traindfX.shape[1])
testX.columns = range(testX.shape[1])
X.head(10), traindf.head()
testX.head()
testX.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17290 entries, 0 to 17289
Data columns (total 20 columns):
date             17290 non-null int32
bedrooms         17290 non-null int64
bathrooms        17290 non-null float64
sqft_living      17290 non-null int64
sqft_lot         17290 non-null int64
floors           17290 non-null float64
waterfront       17290 non-null int64
view             17290 non-null int64
condition        17290 non-null int64
grade            17290 non-null int64
sqft_above       17290 non-null int64
sqft_basement    17290 non-null int64
yr_built         17290 non-null int64
yr_renovated     17290 non-null int64
zipcode          17290 non-null int64
lat              17290 non-null float64
long             17290 non-null float64
sqft_living15    17290 non-null int64
sqft_lot15       17290 non-null int64
price            17290 non-null int64
dtypes: float64(4), int32(1), int64(15)
memory usage: 2.6 MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4323 entries

In [3]:
#shows us our data for y 
y = traindfy['price']
y.head()

0    353000
1    300523
2    435000
3    800000
4    417500
Name: price, dtype: int64

In [4]:
#split the data (necessary for other classifiers)
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [5]:
#checking for correct array sizes
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(12967, 19)
(4323, 19)
(12967,)
(4323,)


In [11]:
#actual linear regression
Lregression = LinearRegression().fit(X.values, y)
print("LinReg Score: ",Lregression.score(X, y))


LinReg Score:  0.7112455445564347


C:\Users\Adam Salyers\Anaconda3\lib\site-packages\sklearn\base.py:434: FutureWarning: The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').
  "multioutput='uniform_average').", FutureWarning)


In [14]:
from sklearn.metrics import mean_squared_error 

from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_score

from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn import metrics


#Predict using cross validation with 10 folds
predictions = Lregression.predict(testX)

#Print the predictions
print(predictions)

#Print mean score fo the cross validation 
scores = cross_val_score(Lregression, X, y, cv=2)
print(np.mean(scores))

'''
The score of accuracy averages at about 0.704 which is lower than when we dont
use cross validation. When I increasee the fold size the accuracy goes down. 
'''

[[2.01410090e+07 4.00000000e+00 2.75000000e+00 ... 2.90000000e+03
  6.75200000e+03 6.31093508e+05]
 [2.01504010e+07 3.00000000e+00 2.25000000e+00 ... 1.62000000e+03
  7.42900000e+03 4.96144286e+05]
 [2.01408190e+07 2.00000000e+00 1.50000000e+00 ... 3.86000000e+03
  1.51081000e+05 6.99677002e+05]
 ...
 [2.01409080e+07 3.00000000e+00 1.50000000e+00 ... 1.84000000e+03
  9.30500000e+03 3.50416123e+05]
 [2.01406100e+07 5.00000000e+00 3.25000000e+00 ... 3.55000000e+03
  1.39170000e+04 1.37951369e+06]
 [2.01410170e+07 3.00000000e+00 1.75000000e+00 ... 1.82000000e+03
  1.81510000e+04 3.58512595e+05]]
0.7099535313823915


C:\Users\Adam Salyers\Anaconda3\lib\site-packages\sklearn\base.py:434: FutureWarning: The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').
  "multioutput='uniform_average').", FutureWarning)
C:\Users\Adam Salyers\Anaconda3\lib\site-packages\sklearn\base.py:434: FutureWarning: The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='unifo

'\nThe score of accuracy averages at about 0.704 which is lower than when we dont\nuse cross validation. When I increasee the fold size the accuracy goes down. \n'

In [ ]:
#Log Regress don't run it takes too long
#Logreg = LogisticRegression(solver='liblinear', random_state=0).fit(X, y)
#print("LogReg Score: ",Logreg.score(X,y))

In [7]:
#***dont run takes foreer will look at it later*** 
#clf = svm.SVC(kernel='linear', C=1).fit(X_train, y_train)
#clf.score(X_test, y_test)

The decision tree seen below is extremely hard to grade since we are looking at the individual house prices.

The Gini score given is always going to be really high, becuase in all liklihood there is only one house per price.

So instead we need to split the price into ranges, or bins, and then re-enter the correct ranges for the prices.

In [8]:
#decision tree classifier
#Dectree = DecisionTreeClassifier(max_depth = 5).fit(X,y)
#Source(export_graphviz(Dectree, out_file = None, feature_names = list(X.columns)))

In [9]:
#Checking the range of the price values
#Pricemin = y_train.min()
#Pricemax = y_train.max()

#print("Min is: ", Pricemin, "Max is: ", Pricemax)

#Creating our range
#Priceranges = np.linspace(Pricemin,Pricemax,15)

#Range_y = y_train.copy()
#Range_y = np.array(Range_y.values)

#for i in range(len(Range_y)):
    #for j in range(len(Priceranges)):
        #if Range_y[i] <= Priceranges[j+1]: 
            #Range_y[i] = str(Priceranges[j])+"-"+str(Priceranges[j+1])
            #break
            

#print(Range_y)

#Now that we have our dataset lets create a new tree and compare!
#Dectree = DecisionTreeClassifier(max_depth = 10).fit(X,Range_y)
#Source(export_graphviz(Dectree, out_file = None, feature_names = list(X.columns)))

#Much Better Right!

In [6]:
#SVM classifier
SVMreg = svm.SVC(kernel='rbf').fit(X_train,y_train)

In [ ]:
predict = SVMreg.predict(X_test)
print(predict)
#print(mean_squared_error(y_test.values,predict))

In [47]:
print(X)

              0  1     2     3      4    5  6  7  8   9    10   11    12  13  \
0      20141002  3  1.75  2190   7021  1.0  0  2  4   7  1390  800  1953   0   
1      20150105  3  2.50  2370   6840  2.0  0  0  3   9  2370    0  1987   0   
2      20140617  2  1.00  1230   3800  1.0  0  0  3   7  1230    0  1928   0   
3      20141124  4  2.25  2510   9963  1.0  0  0  4   9  2200  310  1967   0   
4      20140818  3  1.00  1160   7491  1.0  0  0  4   6  1160    0  1917   0   
...         ... ..   ...   ...    ...  ... .. .. ..  ..   ...  ...   ...  ..   
17285  20140804  3  2.50  1920   3867  2.0  0  0  3   8  1920    0  2005   0   
17286  20150128  4  4.50  3420   7440  3.0  0  0  3   9  3420    0  2014   0   
17287  20150401  3  1.75  1970  54450  1.0  0  0  3   8  1570  400  1980   0   
17288  20140805  3  2.25  1980   8775  1.0  0  0  3   7  1290  690  1959   0   
17289  20140610  5  4.00  3760  28040  2.0  0  0  3  10  3760    0  1983   0   

          14       15       16    17   